**DOC-SIMILARITY MODULE**

In [28]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

file_docs = []

#Tokenize
with open ('demofile.txt',encoding='utf-8') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file_docs.append(line)

print("Number of documents:",len(file_docs))

Number of documents: 4


In [29]:
#tokenize and create dict
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [30]:
#tokenize and create dict
import gensim
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary.token2id)


{'.': 0, 'by': 1, 'calculated': 2, 'calculating': 3, 'distance': 4, 'document': 5, 'is': 6, 'similarity': 7, '(': 8, ')': 9, 'a': 10, 'and': 11, 'angle': 12, 'are': 13, 'as': 14, 'between': 15, 'concept': 16, 'documents': 17, 'given': 18, 'the': 19, 'treated': 20, 'two': 21, 'vectors': 22, 'where': 23, 'words': 24, 'frequency': 25, 'in': 26, 'occurrences': 27, 'of': 28, ':': 29, 'an': 30, 'example': 31, 'let': 32, 's': 33, 'see': 34, '’': 35}


In [31]:
#BOW
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [32]:
#TF-IDF
import numpy as np
tf_idf = gensim.models.TfidfModel(corpus)
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[['.', 0.1], ['by', 0.5], ['calculated', 0.25], ['calculating', 0.5], ['distance', 0.25], ['document', 0.21], ['is', 0.25], ['similarity', 0.5]]
[['.', 0.05], ['calculated', 0.12], ['distance', 0.12], ['document', 0.1], ['is', 0.24], ['(', 0.24], [')', 0.24], ['a', 0.12], ['and', 0.24], ['angle', 0.24], ['are', 0.12], ['as', 0.47], ['between', 0.24], ['concept', 0.24], ['documents', 0.24], ['given', 0.12], ['the', 0.12], ['treated', 0.24], ['two', 0.24], ['vectors', 0.24], ['where', 0.24], ['words', 0.12]]
[['.', 0.07], ['document', 0.14], ['a', 0.17], ['are', 0.17], ['given', 0.17], ['the', 0.17], ['vectors', 0.17], ['words', 0.17], ['frequency', 0.34], ['in', 0.34], ['occurrences', 0.34], ['of', 0.68]]
[[':', 0.38], ['an', 0.38], ['example', 0.38], ['let', 0.38], ['s', 0.38], ['see', 0.38], ['’', 0.38]]


In [33]:
#Similarity Index Measure
# building the index
sims = gensim.similarities.Similarity('./',tf_idf[corpus],
                                        num_features=len(dictionary))


In [34]:
file2_docs = []

with open ('demofile2.txt') as f:
    tokens = sent_tokenize(f.read())
    for line in tokens:
        file2_docs.append(line)

print("Number of documents:",len(file2_docs))  
for line in file2_docs:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc) 
    #update an existing dictionary and create bag of words

Number of documents: 4


In [35]:
# perform a similarity query against the corpus
query_doc_tf_idf = tf_idf[query_doc_bow]
# print(document_number, document_similarity)
print('Comparing Result:', sims[query_doc_tf_idf]) 

Comparing Result: [0.         0.         0.         0.75592893]


In [36]:
#Average Similarity
import numpy as np

sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.75592893


In [37]:
percentage_of_similarity = round(float((sum_of_sims / len(file_docs)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(file_docs))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

Average similarity float: 0.18898223340511322
Average similarity percentage: 18.898223340511322
Average similarity rounded percentage: 19


In [38]:
file2_docs

['Document similarity is calculated by calculating document distance.',
 'Document distance is a concept where words(documents) are treated as vectors and is calculated as the angle between two given document vectors.',
 'Document vectors are the frequency of occurrences of words in a given document.',
 'Letâ€™s see an example:']

**DIRECT**

In [39]:
avg_sims = [] # array of averages

# for line in query documents
for line in file2_docs:
    
    # tokenize words
    query_doc = [w.lower() for w in word_tokenize(line)]
    # create bag of words
    query_doc_bow = dictionary.doc2bow(query_doc)
    # find similarity for each document
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print (document_number, document_similarity)
    print('Comparing Result:', sims[query_doc_tf_idf]) 
    # calculate sum of similarities for each query doc
    sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
    # calculate average of similarity for each query doc
    avg = sum_of_sims / len(file_docs)
    # print average of similarity for each query doc
    print(f'avg: {sum_of_sims / len(file_docs)}')
    # add average values into array
    avg_sims.append(avg)  
# calculate total average
total_avg = np.sum(avg_sims, dtype=np.float)
# round the value and multiply by 100 to format it as percentage
percentage_of_similarity = round(float(total_avg) * 100)
# if percentage is greater than 100
# that means documents are almost same
if percentage_of_similarity >= 100:
    percentage_of_similarity = 100

Comparing Result: [1.0000001  0.14399059 0.03662721 0.        ]
avg: 0.29515448212623596
Comparing Result: [0.14399059 1.         0.1570631  0.        ]
avg: 0.32526344060897827
Comparing Result: [0.03662721 0.1570631  1.         0.        ]
avg: 0.29842257499694824
Comparing Result: [0.         0.         0.         0.75592893]
avg: 0.18898223340511322


In [40]:
sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)


0.75592893
